In [1]:
import torch
import numpy as np
import pandas as pd
import codecs
import re
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

### 0.Preprocessing

In [2]:
# Encoding in windows-1252, utf-8 generate error on some char
file = codecs.open("train_all.label", "r+","windows-1252")
train_data = []
list_of_words = []
for line in file.readlines():
    for word in line.split():
        list_of_words.append(word)
    train_data.append(line)

In [3]:
# Divided file into 2 list:
# questions = list of questions 
# labels = list of labels

questions = []
labels = []

(train_data[0]).split()[0]
for string in train_data:
    labels.append(string.split()[0])
    questions.append(string.split()[1:])
len(questions)

15452

In [4]:
# Lemmatisation
# Je travaille sur list_of_words
def lemm(phrase):
    lemmer = WordNetLemmatizer()
    lemmed = []
    for word in phrase:
        lemmed.append(lemmer.lemmatize(word, pos="v"))
    return lemmed

In [5]:
list_of_lemms = lemm(list_of_words)

In [6]:
# Method to add tags begin and end to phrases list.
# /!\ 
# WARNING : this method need to be executed only ONE time.
# /!\
def add_tag():
    for i in range(0, len(questions)):
        questions[i].insert(0, '<bos>')
        questions[i].append('<eos>')
# add_tag()

In [7]:
# Using sklearn to get the vocabulary
vectorizer = CountVectorizer()
str_questions = []
for quest in questions:
    str_questions.append(' '.join(quest))
output = vectorizer.fit(str_questions)

In [8]:
# This is the vocabulary dict.
vocabulary = output.vocabulary_
word_list = list(vocabulary.keys())
words_array = np.array(word_list)

In [9]:
# Integer encoding with OneHotEncoder
words_tre = words_array.reshape(len(words_array),1)
one_hot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = one_hot_encoder.fit_transform(words_tre)
print(onehot_encoded)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
# Creating a dictionnary of word and it one hot array
words_onehoted = {}
for i in range(0, len(words_array)):
    words_onehoted[word_list[i]] = onehot_encoded[i]

In [15]:
# Function to get the one hot list for a choosed word.
def get_onehot_word(word):
    out = []
    for key, value in words_onehoted.items():
        if key == word:
            out = list(value)
    return out

In [16]:
get_onehot_word('bos')

[]

### 1.First RNN Model

In [13]:
import torch.nn as nn
